In [ ]:
from __future__ import print_function
import fitz, sys

def split_pdf(finput):
    doc = fitz.open(finput)
    page = doc.loadPage(0)
    if int(page.MediaBox[2]) > 800.0:
        res = fitz.open() 

        for spage in doc:
            r = spage.rect  
            d = fitz.Rect(spage.CropBoxPosition, spage.CropBoxPosition)  
    
            r1 = fitz.Rect(0, 0, r.width*0.5, r.height)
            r2 = fitz.Rect(r.width*0.5, 0, r.width, r.height)
            rect_list = [r1, r2]
    
            for rx in rect_list:  
                rx += d 
                page = res.newPage(-1, width = rx.width, height = rx.height)
                page.showPDFpage(page.rect, doc, spage.number, clip = rx)

        res.save(doc.name[:-3]+'_sp.pdf', garbage = 3, deflate = True)
        return doc.name[:-3]+'_sp.pdf'

In [ ]:
import pdftotext
import os

def text22text(finput):
    
    fout = os.path.join(os.getcwd(), 'output.txt')
    
    if '.pdf' in finput:
         
        with open(finput, "rb") as f:
            pdf = pdftotext.PDF(f)

        # Save all text to a txt file.
        with open('output.txt', 'w',-1,'utf-8') as f:
            f.write("\n\n".join(pdf))
    
        with open('output.txt', 'r',-1,'utf-8') as f:
            text=f.read()
            
    elif '.HWP' or '.hwp' in finput:
        
        hwp2txt_cmd = r'hwp5txt --output={} {}'.format(fout, finput)
        os.system(hwp2txt_cmd)
        with open(fout,'r',encoding='UTF8') as f:
            text=f.read()
            
    return text

In [ ]:
import glob
from PIL import Image
from pytesseract import *
#from PyPDF2 import PdfFileReader
from pdf2image import convert_from_bytes
import cv2

def image22text(finput, seqnum):
    img_name = os.path.join(r'C:\pgm_file', 'img_data', seqnum) 
    os.mkdir(img_name)
    images = convert_from_bytes(open(finput, 'rb').read(), dpi=300, output_folder = img_name, fmt="png")
        
    text = ''
    for img in os.listdir(img_name):
        img = cv2.imread(os.path.join(img_name, img))
        img = cv2.resize(img, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_CUBIC)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 85, 11)
        
        custom_config = r'-l kor -c tessedit_char_blacklist=ABCDEFGHIJKLMNOPQRSTUVWXYZ --psm 6'
        temp = pytesseract.image_to_string(thresh, config = custom_config, lang='kor')
        text += temp
    return text

In [ ]:
import os
import pandas as pd 

tlist = []
xname = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', f"{'xlsx_#1_file_list'}.xlsx")
df = pd.read_excel(xname)

for i in range(2, len(df)):
    
    fpath = df.loc[i, 'path']
    
    if 'pdf' in fpath:
        fpath = split_pdf(fpath)

    text = text22text(fpath)
    if text.isspace():
        text = image22text(fpath, df.loc[i, 'sequm'])
    
    tlist.append(text)
    print(n)

df['text'] = tlist
x2name = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', f"{'xlsx_#2_text'}.xlsx")
df.to_excel(x2name)